<a href="https://colab.research.google.com/github/zigzager/SF/blob/main/generador_intervalo_v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# --- APARTADO 1: Importaciones ---
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import uuid
# --- FIN APARTADO 1 ---
ruta = r"C:\Users\marou\Desktop\XAUUSD_M1_202301110812_202511142349.csv"
# --- APARTADO 2: Definición de la Función ---
def generar_imagen_intervalo(ruta_csv, formato_fecha): # <--- ¡CAMBIO AQUÍ!
    """
    VERSIÓN 2.3: Genera un gráfico de barras minimalista (Máximo-Mínimo)
    con un punto para el precio de Cierre.

    Ahora acepta un formato de fecha para ser flexible (M1 o H1).
    """

    print("\n--- Generador de Gráficos (Barras Minimalistas) ---")
# --- FIN APARTADO 2 ---
    # --- APARTADO 3: Carga y Procesamiento del CSV ---
    try:
        separador = '\t'

        df = pd.read_csv(
            ruta_csv,
            sep=separador,
            parse_dates={'Datetime': ['<DATE>', '<TIME>']}
        )

        # Renombramos las 3 columnas que necesitamos
        df.rename(columns={
            '<CLOSE>': 'Close',
            '<HIGH>': 'High',
            '<LOW>': 'Low'
        }, inplace=True)

        # Asegurarse de que Datetime sea el índice
        df.set_index('Datetime', inplace=True)

        # Mantenemos las 3 columnas que vamos a dibujar
        df = df[['High', 'Low', 'Close']]

        print(f"CSV '{ruta_csv}' cargado y procesado (H/L/C).")
        print(f"Datos disponibles desde {df.index.min()} hasta {df.index.max()}")

    except FileNotFoundError:
        print(f"Error: El archivo CSV no se encontró en: {ruta_csv}")
        return
    except Exception as e:
        print(f"Error al cargar o procesar el CSV: {e}")
        print("Asegúrate de que las columnas '<DATE>', '<TIME>', '<HIGH>', '<LOW>' y '<CLOSE>' existen.")
        return
    # --- FIN APARTADO 3 ---
    # --- APARTADO 4: Interfaz de Usuario (Pedir Fechas) ---
    # formato_fecha = '%Y-%m-%d %H:%M'  <-- YA NO DEFINIMOS ESTO AQUÍ

    while True:
        try:
            # --- ¡CAMBIO AQUÍ! ---
            # Usa la variable "formato_fecha" que le pasamos a la función
            print(f"\nIntroduce las fechas en formato: {formato_fecha}")

            # Modificamos los ejemplos para que sean dinámicos
            if formato_fecha == '%Y-%m-%d %H:%M':
                ejemplo_inicio = "2024-10-20 14:30"
                ejemplo_fin = "2024-10-20 16:00"
            else: # Asumimos H1
                ejemplo_inicio = "2024-10-20 14"
                ejemplo_fin = "2024-10-20 20"

            fecha_inicio_str = input(f"Introduce fecha/hora de INICIO (ej: {ejemplo_inicio}): ")
            fecha_inicio = datetime.strptime(fecha_inicio_str, formato_fecha)

            fecha_fin_str = input(f"Introduce fecha/hora de FIN (ej: {ejemplo_fin}):   ")
            fecha_fin = datetime.strptime(fecha_fin_str, formato_fecha)
            # --- FIN DEL CAMBIO ---

            if fecha_fin <= fecha_inicio:
                print("Error: La fecha de fin debe ser posterior a la fecha de inicio.")
                continue

            break
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Debe ser: {formato_fecha}")
# --- FIN APARTADO 4 ---
# --- APARTADO 5: Filtrado de Datos ---
    print(f"Filtrando datos desde {fecha_inicio} hasta {fecha_fin}...")
    datos_intervalo = df.loc[fecha_inicio : fecha_fin].copy()

    if datos_intervalo.empty:
        print(f"No se encontraron datos para el intervalo especificado.")
        return
# --- FIN APARTADO 5 ---
    # --- APARTADO 6: Generación del Gráfico (El "Dibujo") ---
    fig, ax = plt.subplots(figsize=(6, 4), dpi=100)  # Imagen de 600x400 píxeles

    # Lógica para ajustar el tamaño del punto y el grosor de la línea según la densidad de datos
    numero_de_puntos = len(datos_intervalo)
    if numero_de_puntos > 300:  # Más de 5 horas de datos M1
        marker_size = 1
        line_width = 0.5
    else:
        marker_size = 2
        line_width = 1

    # 1. Dibujar las líneas verticales de Máximo a Mínimo
    ax.vlines(datos_intervalo.index, ymin=datos_intervalo['Low'], ymax=datos_intervalo['High'], color='black', linewidth=line_width)
    # 2. Dibujar los puntos de Cierre con el tamaño dinámico
    ax.plot(datos_intervalo.index, datos_intervalo['Close'], 'o', color='black', markersize=marker_size)
    # Configuración de fondo blanco y sin ejes
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    ax.axis('off')
    ax.margins(0, 0)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
# --- FIN APARTADO 6 ---
    # --- APARTADO 7: Guardado de la Imagen ---

    # 1. Crear el nombre base (igual que antes)
    base_nombre = f"{fecha_inicio.strftime('%Y%m%d%H%M')}_{fecha_fin.strftime('%Y%m%d%H%M')}"

    # 2. Lógica para crear el 'xyz' aleatorio
    # Usamos uuid4() para generar un ID único aleatorio
    # .hex[:6] lo convierte en una cadena corta de 6 caracteres (letras minúsculas y números)
    xyz_aleatorio = uuid.uuid4().hex[:6]

    # 3. Construir el nombre de archivo final
    nombre_archivo = f"{base_nombre}_{xyz_aleatorio}.png"

    # 4. Guardar el archivo
    # (Ya no necesitamos el bucle 'while' porque el uuid es estadísticamente único)
    plt.savefig(nombre_archivo, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig) # Liberar memoria

    print(f"\n¡Éxito! Gráfico (Mecha y Cierre) guardado como: '{nombre_archivo}'")
# --- FIN APARTADO 7 ---
# --- APARTADO 8: Bloque de Ejecución Principal ---
if __name__ == "__main__":

    # --- ¡CAMBIO AQUÍ! ---
    # Define las rutas de TUS DOS archivos (súbelos a Colab)
    ruta_m1 = "XAUUSD_M1_202301110812_202511142349.csv"
    ruta_h1 = "XAUUSD_H1_...[tu_archivo_h1]....csv" # <-- CAMBIA ESTO
    # --- FIN DEL CAMBIO ---

    # Bucle principal del Menú
    while True:
        print("\n--- MENÚ PRINCIPAL ---")
        print("¿Qué datos quieres visualizar?")
        print("1: Gráfica de 1 Minuto (M1)")
        print("2: Gráfica de 1 Hora (H1)")
        print("q: Salir del programa")

        opcion = input("Elige una opción (1, 2, q): ")

        if opcion == '1':
            ruta_a_usar = ruta_m1
            formato_a_usar = '%Y-%m-%d %H:%M' # Formato con minutos

        elif opcion == '2':
            ruta_a_usar = ruta_h1
            formato_a_usar = '%Y-%m-%d %H' # Formato solo con hora

        elif opcion == 'q' or opcion == 'Q':
            break # Sale del bucle principal

        else:
            print("Opción no válida. Por favor, elige 1, 2 o q.")
            continue # Vuelve al inicio del menú

        # Llama a la función principal con los ajustes correctos
        generar_imagen_intervalo(ruta_a_usar, formato_a_usar)

        # Pregunta si quiere generar OTRA imagen con la MISMA temporalidad
        while True:
            continuar = input("\n¿Quieres generar otra imagen de esta temporalidad? (s/n): ").lower()
            if continuar == 's':
                generar_imagen_intervalo(ruta_a_usar, formato_a_usar)
            else:
                break # Vuelve al menú principal

    print("Programa terminado.")
# --- FIN APARTADO 8 ---



--- Generador de Gráficos (Barras Minimalistas) ---


/tmp/ipython-input-4287514032.py:22: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(


CSV 'XAUUSD_M1_202301110812_202511142349.csv' cargado y procesado (H/L/C).
Datos disponibles desde 2023-01-11 08:12:00 hasta 2025-11-14 23:49:00

Introduce las fechas en formato: %Y-%m-%d %H:%M
Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): 2025-11-14 11:21
Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   2025-11-14 23:21
Filtrando datos desde 2025-11-14 11:21:00 hasta 2025-11-14 23:21:00...

¡Éxito! Gráfico (Mecha y Cierre) guardado como: '202511141121_202511142321_1209d3.png'

¿Quieres generar otra imagen? (s/n): n
Programa terminado.
